# Практическая работа №8. Методы защиты от атак на модели ИИ

Выполнил Сердюков Матвей, ББМО-01-23

## Цель задания

Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных,
моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак,
изученных ранее.

## Задачи

1. Изучить и реализовать защиту модели с помощью тренировок на противоречивых примерах
(Adversarial Training).
2. Реализовать метод защиты на основе градиентной маскировки.
3. Использовать регуляризацию и нормализацию для повышения устойчивости модели.
4. Проверить эффективность методов защиты против атак FGSM, PGD и GAN-based атак.
5. Оценить улучшение точности защищенной модели на противоречивых примерах.

## Защита с помощью Adversarial Training

In [4]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist

# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image

# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
    adversarial_images = []
    for i in range(len(images)):
        image = tf.convert_to_tensor(images[i].reshape((1, 28, 28, 1)), dtype=tf.float32)
        label = tf.convert_to_tensor(labels[i].reshape((1, 10)), dtype=tf.float32)
        with tf.GradientTape() as tape:
            tape.watch(image)
            prediction = model(image)
            loss = tf.keras.losses.categorical_crossentropy(label,
            prediction)
        gradient = tape.gradient(loss, image)
        adversarial_image = fgsm_attack(image, epsilon, gradient)
        adversarial_images.append(adversarial_image.reshape(28, 28))
    return np.array(adversarial_images)

# Создание модели
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
    for epoch in range(5): # Короткое обучение для демонстрации
        for i in range(0, len(train_images), 32):
            batch_images = train_images[i:i+32]
            batch_labels = train_labels[i:i+32]
            
            # Генерация противоречивых примеров для текущей партии данных
            adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
            
            # Объединение оригинальных и противоречивых примеров
            combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
            combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
            
            # Обучение на комбинированных данных
            model.train_on_batch(combined_images, combined_labels)

# Инициализация модели
model = create_model()

# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1)

# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

## Градиентная маскировка

In [5]:
# Реализация градиентной маскировки
# Для демонстрации мы можем использовать специальные функции активации
from tensorflow.keras.layers import Activation

# Обновление модели для градиентной маскировки
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus') # Используем softplus вместо softmax для градиентной маскировки
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4613 - loss: 1.9016 
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8312 - loss: 0.7994
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8789 - loss: 0.4886
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9263 - loss: 0.3106 
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9494 - loss: 0.2451 


## Регуляризация и нормализация для повышения устойчивости

In [6]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3364 - loss: 4.0272 
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7400 - loss: 2.2092
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8278 - loss: 1.6931
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8731 - loss: 1.4790
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8888 - loss: 1.3142


## Оценка моделей на противоречивых примерах

In [7]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')

# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model, test_images, test_labels, epsilon=0.1)

# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, test_labels)
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step - accuracy: 0.4999 - loss: 1.4629
Accuracy of protected model on adversarial examples: 0.5544999837875366


In [8]:
# Загрузка атакованной модели
masked_model = tf.keras.models.load_model('masked_model.h5')

# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images, test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step - accuracy: 0.3964 - loss: 1.7908
Accuracy of masked model on adversarial examples: 0.44179999828338623


In [9]:
# Загрузка атакованной модели
regularized_model = tf.keras.models.load_model('regularized_model.h5')

# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images, test_labels)
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.5351 - loss: 2.2939
Accuracy of masked model on adversarial examples: 0.5932000279426575


## Вывод

В ходе работы были применены три различных метода защиты моделей ИИ от атак с использованием противоречивых примеров. Исследованные методы в различной степени делают модель более устойчивой к подобным атакам, посколько точность при их применении снизилась не настолько же сильно, насколько она падала при использовании незащищённых моделей в прошлых практических работах.

Используя полученные результаты, можно составить следующий рейтинг методов защиты:

1. Регуляризация и нормализация (59%)
2. Adversarial Training (55%)
3. Градиентная маскировка (44%)